In [1]:
from google.colab import files
uploaded = files.upload()

Saving vgsales.csv to vgsales.csv


In [2]:
import pandas as pd
# Leer archivo CSV
df = pd.read_csv('vgsales.csv')

# Mostrar las primeras filas del dataset para asegura de que se cargó correctamente
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
# -------------------------------
# PENALIZACIONES L1 Y L2
# -------------------------------

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler


# Cargar el dataset
data_encoded = pd.read_csv('vgsales.csv')

# Convertir espacios vacíos a NaN
data_encoded.replace("", np.nan, inplace=True)

# Imputar valores faltantes en las columnas de ventas
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']] = imputer.fit_transform(
    data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
)

# Normalizar las columnas de ventas
scaler = MinMaxScaler()
data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']] = scaler.fit_transform(
    data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
)

# ------------------------------
# SELECCIONAR CARACTERÍSTICAS Y VARIABLE OBJETIVO
# ------------------------------
X = data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].values
y = data_encoded['Global_Sales'].values

# ------------------------------
# APLICAR PENALIZACIONES (L1 Y L2)
# ------------------------------
# # Modelo Ridge (L2)
# ridge = Ridge(alpha=1.0)
# ridge.fit(X, y)

# # Modelo Lasso (L1)
# lasso = Lasso(alpha=0.0001)
# lasso.fit(X, y)


# -------------------------------
# Función de Regresión Ridge (L2)
# -------------------------------
def regresion_ridge(X, y, alpha):
    # Número de características
    num_caracteristicas = X.shape[1]
    # Crear una matriz identidad de tamaño num_caracteristicas x num_caracteristicas
    I = np.eye(num_caracteristicas)

    # Calcular X transpuesto (X^T)
    X_transpuesta = X.T

    # Calcular A = X^T * X + alpha * I
    A = np.dot(X_transpuesta, X) + alpha * I

    # Calcular b = X^T * y
    b = np.dot(X_transpuesta, y)

    # Resolver A * coeficientes = b para obtener los coeficientes
    coeficientes = np.linalg.solve(A, b)

    return coeficientes

# -------------------------------
# Función de Regresión Lasso (L1)
# -------------------------------
def regresion_lasso(X, y, alpha, num_iteraciones=1000, tasa_aprendizaje=0.01):
    # Inicializar coeficientes en ceros
    coeficientes = np.zeros(X.shape[1])

    for _ in range(num_iteraciones):
        # Calcular las predicciones
        y_pred = np.dot(X, coeficientes)

        # Calcular el error
        error = y_pred - y

        # Calcular el gradiente
        gradiente = np.dot(X.T, error) / len(y)

        # Actualizar los coeficientes
        # Agregar la penalización L1
        coeficientes -= tasa_aprendizaje * (gradiente + alpha * np.sign(coeficientes))

    return coeficientes

# ------------------------------
# APLICAR PENALIZACIONES (L1 Y L2)
# ------------------------------
alpha_ridge = 1.0  # Valor de alpha para Ridge
alpha_lasso = 0.0001 # Valor de alpha para Lasso

# Calcular coeficientes con Ridge
coef_ridge = regresion_ridge(X, y, alpha_ridge)
print("Coeficientes de Ridge:", coef_ridge)

# Calcular coeficientes con Lasso
coef_lasso = regresion_lasso(X, y, alpha_lasso)
print("Coeficientes de Lasso:", coef_lasso)



Coeficientes de Ridge: [0.42991647 0.31179515 0.1398865  0.16869881]
Coeficientes de Lasso: [0.0028728  0.002264   0.00294261 0.00178807]
